# Working with Redcap: API and Data Wrangling
- Intro to Pandas library
- Intro to Redcapy library
- Intro to Twilio API

- The beginning is a highly abbreviated version of what is available here: https://pandas.pydata.org/pandas-docs/stable/10min.html

### Import libraries
- pd and np are standard conventions
- Pandas is the standard library for working with data
 - Ref: https://pandas.pydata.org/pandas-docs/stable/index.html
- Numpy is the standard library for array processing and scientific computing
 - Ref: http://www.numpy.org/

In [ ]:
import pandas as pd
import numpy as np

from IPython.core.debugger import set_trace

### Series is a basic pandas data structure
- Element types are floats

In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

### Series type is Series

In [ ]:
type(s)  

### A DataFrame is comprised of series and an index

In [ ]:
dates = pd.date_range('20130101', periods=6)
dates

### Create a DataFrame with random numbers

In [ ]:
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

### Check data type of each column

In [ ]:
df.dtypes

### View the beginning or end of data with head or tail
- default is up to 5 records

In [ ]:
df.head(3)  # first three rows

### Last row with tail

In [ ]:
df.tail(1)

### Extract a column of a DataFrame using an attribute
- Returns a Series
- Existing DataFrame column names are available with TAB completion

In [ ]:
df.A

### Or extract using bracket notation

In [ ]:
df['A']

### Bracket is useful when using a variable for the column name
- Note: Bracket notation is required when creating a new DataFrame column

In [ ]:
my_col = 'A'
df[my_col]

### Extract multiple columns by passing a list to the brackets
- Returns a DataFrame

In [ ]:
df[['A', 'B']]

### Extract the index from the DataFrame

In [ ]:
df.index  # or convert to a list by: list(df.index)

### Now, create a new DataFrame with different series types

In [ ]:
df2 = pd.DataFrame({ 'A' : 1.,
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D' : np.array([3] * 4, dtype='int32'),
                    'E' : pd.Categorical(["test","train","test","train"]),
                    'F' : 'foo' })
df2

### Check Column Types

In [ ]:
df2.dtypes

### Transpose Data

In [ ]:
df2.T

### Apply an operation to a column of the DataFrame
- Double each element of column D

In [ ]:
df2.D * 2

In [ ]:
df2.F + 'bar'

### Apply an operation to a column, contingent on a value in another column

In [ ]:
df2['D2'] = df2.apply(lambda row: row.D * 2 if row.E == 'test' else row.D, axis=1)
df2

### Apply an operation to a Series, not contingent on a value in another column

In [ ]:
df2['D3'] = df2.D2.apply(lambda x: x * 3 if x == df2.D2.min() else x)
df2['D3'] 

## Create pre-randomization data that resembles a JSON export from Redcap
- Every record always contains
 - record_id
 - redcap_event_name
- Data from a repeating instrument will also contain
 - redcap_repeat_instrument
 - redcap_repeat_instance
 
 
- Data is intuitively stacked by record id and event and repeating instance

In [ ]:
rc_import = [
    {'record_id': '1',
     'redcap_event_name': 'baseline_arm_1',
     'part_first_name': 'Ann',
     'part_last_name': 'Apple',
     'part_age': '3',
     'part_mobile_number': '415-555-1212',
    },
    {'record_id': '2',
     'redcap_event_name': 'baseline_arm_1',
     'part_first_name': 'Bob',
     'part_last_name': 'Barley',
     'part_age': '4',
     'part_mobile_number': '415-555-1212',
    },
    {'record_id': '4',
     'redcap_event_name': 'baseline_arm_1',
     'part_first_name': 'Don',
     'part_last_name': 'Dill',
     'part_age': '6',
     'part_mobile_number': '415-555-1212',
    },
    {'record_id': '5',
     'redcap_event_name': 'baseline_arm_1',
     'part_first_name': 'Eve',
     'part_last_name': 'Eggs',
     'part_age': '5',
     'part_mobile_number': '415-555-1212',
    },    
]

rci_df = pd.DataFrame(rc_import)
rci_df

### Define Redcapy instances for importing and exporting
- In this example, the same tokens are used for both instances, but using different tokens can be useful for:
 - Merging data from multiple projects
 - Testing imports by exporting from production project and importing into a development copy

In [ ]:
from redcap.redcapy import Redcapy
import os

redcap_token = os.environ['REDCAP_API_CAPS_DEMO']
redcap_url = os.environ['REDCAP_URL']

rce = Redcapy(api_token=redcap_token, redcap_url=redcap_url)
rci = Redcapy(api_token=redcap_token, redcap_url=redcap_url)

data_to_import = rci_df.to_json(orient='records')
data_to_import, type(data_to_import)


In [ ]:
def import_to_redcap(redcap_instance, df_to_upload):
    import_success_count = 0
    import_attempt_count = 0
    
    for i, row in df_to_upload.iterrows():
        record_to_upload = row.to_json(orient='columns')
#         print(record_to_upload)
        import_return = redcap_instance.import_records(data_to_upload=record_to_upload)  # returns {'count': 1} if successful
        
        import_success_count += 1 if 'count' in import_return and import_return['count'] == 1 else 0
        import_attempt_count += 1
        
    return import_success_count, import_attempt_count

success_count, total_count = import_to_redcap(rci, rci_df)
success_count, total_count, success_count == total_count

### Try importing data into an event which doesn't contain fields

In [ ]:
rc_bad = [
    {'record_id': '1',
     'redcap_event_name': 'baseline_arm_1',
     'redcap_repeat_instrument': '',
     'redcap_repeat_instance': '',
     'exam_visit_date': '2017-08-01',
     'exam_num_teeth': '6',
     'exam_active_caries_count': 3,
    },
    {'record_id': '2',
     'redcap_event_name': 'baseline_arm_1',
     'redcap_repeat_instrument': '',
     'redcap_repeat_instance': '',     
     'exam_visit_date': '2017-08-02',
     'exam_num_teeth': '7',
     'exam_active_caries_count': 4,
    },

    {'record_id': '4',
     'redcap_event_name': 'baseline_arm_1',
     'redcap_repeat_instrument': '',
     'redcap_repeat_instance': '',     
     'exam_visit_date': '2017-08-04',
     'exam_num_teeth': '8',
     'exam_active_caries_count': 5,
    },
    {'record_id': '5',
     'redcap_event_name': 'baseline_arm_1',
     'redcap_repeat_instrument': '',
     'redcap_repeat_instance': '',     
     'exam_visit_date': '2017-08-03',
     'exam_num_teeth': '8',
     'exam_active_caries_count': 5,
    },    
]

bad_df = pd.DataFrame(rc_bad)

try:
    success_count, total_count = import_to_redcap(rci, bad_df)
    print(success_count, total_count, success_count == total_count)
except Exception as e:
    print(e)

### First, export the data to extract the arm info after randomization

In [ ]:
rc_export = rce.export_records(rawOrLabel='raw')
rc_export

### Create a mapping of record ids to their respective arms

In [ ]:
id_to_arm = dict(map(lambda x: (x['record_id'], x['rand_color']),  rc_export))

# Use below if post-randomization data already exists in Redcap
id_to_arm = dict(map(lambda x: (x['record_id'], x['rand_color']), 
                     list(filter(lambda d: d['redcap_event_name'] == 'baseline_arm_1' 
                                 and not d['redcap_repeat_instrument'], rc_export))))

id_to_arm

### Create Post-randomization test data
- However, depending on the randomization schedule, it is not clear which arm to use without checking each ID in Redcap


In [ ]:
post_rand_import = [ 
    {'record_id': '1',
     'redcap_event_name': 'baseline_arm_1',
     'rand_date': '2017-08-01',
    },
    {'record_id': '2',
     'redcap_event_name': 'baseline_arm_1',    
     'rand_date': '2017-08-02',
    },
    {'record_id': '5',
     'redcap_event_name': 'baseline_arm_1',    
     'rand_date': '2017-08-03',
    },   
    {'record_id': '1',
     'redcap_event_name': '6_month_arm_',      # Note events are incomplete
     'exam_visit_date': '2018-02-01',
     'exam_num_teeth': '11',
    },
    {'record_id': '2',
     'redcap_event_name': '6_month_arm_',  
     'exam_visit_date': '2018-02-02',
     'exam_num_teeth': '25',  
    },
    {'record_id': '5',
     'redcap_event_name': '6_month_arm_',   
     'exam_visit_date': '2018-02-03',
     'exam_num_teeth': '28', 
    },
    {'record_id': '1',
     'redcap_event_name': '6_month_arm_',
     'redcap_repeat_instrument': 'phone_follow_up',
     'redcap_repeat_instance': '1',
     'pfu_contact_date': '2018-03-01', 
     'pfu_call_outcome': 1,
     'pfu_ae_reported': '',
     'pfu_ae_grade': '',
     'pfu_ae_attribution': '',
     'pfu_ae_serious': '',     
    },
    {'record_id': '1',
     'redcap_event_name': '6_month_arm_',
     'redcap_repeat_instrument': 'phone_follow_up',
     'redcap_repeat_instance': '1',     
     'pfu_contact_date': '2018-03-15',  
     'pfu_call_outcome': 1,
     'pfu_ae_reported': '',
     'pfu_ae_grade': '',
     'pfu_ae_attribution': '',
     'pfu_ae_serious': '',      
    },
    {'record_id': '1',
     'redcap_event_name': '6_month_arm_',
     'redcap_repeat_instrument': 'phone_follow_up',
     'redcap_repeat_instance': '1',     
     'pfu_contact_date': '2018-03-30',
     'pfu_call_outcome': 2,
     'pfu_ae_reported': 1,
     'pfu_ae_grade': 1,
     'pfu_ae_attribution': 5,
     'pfu_ae_serious': 0,
    },
    {'record_id': '2',
     'redcap_event_name': '6_month_arm_',
     'redcap_repeat_instrument': 'phone_follow_up',
     'redcap_repeat_instance': '2',     
     'pfu_contact_date': '2018-04-02',   
     'pfu_call_outcome': 1,
     'pfu_ae_reported': '',
     'pfu_ae_grade': '',
     'pfu_ae_attribution': '',
     'pfu_ae_serious': '',      
    },
    {'record_id': '2',
     'redcap_event_name': '6_month_arm_',
     'redcap_repeat_instrument': 'phone_follow_up',
     'redcap_repeat_instance': '2',     
     'pfu_contact_date': '2018-04-03',  
     'pfu_call_outcome': 1,
     'pfu_ae_reported': '',
     'pfu_ae_grade': '',
     'pfu_ae_attribution': '',
     'pfu_ae_serious': '',      
    },   
    {'record_id': '5',
     'redcap_event_name': '6_month_arm_',
     'redcap_repeat_instrument': 'phone_follow_up',
     'redcap_repeat_instance': '1',     
     'pfu_contact_date': '2018-05-02',    
     'pfu_call_outcome': 2,
     'pfu_ae_reported': 1,
     'pfu_ae_grade': 3,
     'pfu_ae_attribution': 1,
     'pfu_ae_serious': 1,
    },      
    {'record_id': '1',
     'redcap_event_name': '12_month_arm_',
     'redcap_repeat_instrument': '',
     'redcap_repeat_instance': '',   
     'exam_visit_date': '2018-08-02',
     'exam_num_teeth': '18',    
    },
]

post_rand_import = [d for d in post_rand_import]

### Iterate the list and update the event name

In [ ]:
new_post_rand_import = []

for d in post_rand_import:
    d_new = {}
    [d_new.update({k: v + id_to_arm[d['record_id']]}) 
                 if k == 'redcap_event_name' and v[-1] == '_' else d_new.update({k: v}) for k, v in d.items()]
    new_post_rand_import.append(d_new)
    
new_post_rand_import

### Repeat above using a DataFrame

In [ ]:
post_rand_df = pd.DataFrame(post_rand_import)
post_rand_df.redcap_event_name = post_rand_df.apply(lambda row: row.redcap_event_name + id_to_arm[row.record_id] 
                                                    if row.redcap_event_name[-1] == '_' else row.redcap_event_name,
                                                    axis=1)
post_rand_df

### Sort by ID and event
- To store sorted results, reassign result to DataFrame or use option inplace=True

In [ ]:
post_rand_df.sort_values(by=['record_id', 'redcap_event_name', 'redcap_repeat_instance'])

# Option 1:
# post_rand_df = post_rand_df.sort_values(by=['record_id', 'redcap_event_name', 'redcap_repeat_instance'])

# Option 2:
# post_rand_df.sort_values(by=['record_id', 'redcap_event_name', 'redcap_repeat_instance'], inplace=True)

### Import Post-Randomization Data

In [ ]:
success_count, total_count = import_to_redcap(rci, post_rand_df)
success_count, total_count, success_count == total_count

### Check all the types of the columns

In [ ]:
post_rand_df.dtypes

### Check type cell by cell
- It is important to keep in mind what type you may be dealing with when applying operations to rows or columns

In [ ]:
post_rand_df.applymap(type)

### If you need to view or use data without NaNs
- For a series

In [ ]:
post_rand_df.redcap_repeat_instance.replace(np.NaN, '')

- For a DataFrame

In [ ]:
post_rand_df.apply(lambda col: col.replace(np.NaN, ''))
# post_rand_df.apply(lambda col: col.replace(np.NaN, '', inplace=True))

- Or, more succinctly

In [ ]:
post_rand_df.fillna('')

### Try computing the mean number of teeth

In [ ]:
try:
    print(post_rand_df.exam_num_teeth.mean())
except TypeError as e:
    print(e)

### Try converting to int

In [ ]:
try:
    print(post_rand_df.exam_num_teeth.apply(int))
except ValueError as e:
    print(e)

In [ ]:
post_rand_df.exam_num_teeth = post_rand_df.exam_num_teeth.apply(lambda x: int(x) if isinstance(x, str) else x)
post_rand_df.exam_num_teeth

### Now compute the mean
- Note above that ints are converted to floats due to column type containing NaNs


In [ ]:
post_rand_df.exam_num_teeth.mean()

### Export a clean copy of all data

In [ ]:
new_export = rce.export_records()
new_df = pd.DataFrame(new_export)
new_df

### To filter data by event and fields
- For performance reasons, filtering is especially important once the project accrues data

In [ ]:
rce.export_records(events=['baseline_arm_1'], fields=['record_id', 'rand_date'])

### Data is exported from Redcap as str
- So type conversion is typically an essential component of working with exported data

In [ ]:
new_df.applymap(type)

## Export the Data Dictionary


In [ ]:
dd = rce.export_data_dictionary()
dd_df = pd.DataFrame(dd)
dd_df

### Define a function to search field names or labels for substring from the Data Dictionary
- Case insensitive

In [ ]:
def search_dd(df, search_str):
    return df[df.apply(lambda row: True if search_str.lower() in row.field_label.lower() 
#                        or search_str.lower() in row.form_name.lower()
                        or search_str.lower() in row.field_name.lower() else False, axis=1)]

search_dd(dd_df, 'date')

### Define a function to show all fields associated with a particular form (from the Data Dictionary)

In [ ]:
def get_all_form_fields(df, form_name):
    return df[df.apply(lambda row: True if row.form_name == form_name else False, axis=1)]

get_all_form_fields(dd_df, 'phone_follow_up')

### Define a function to search column names for a substring
- Useful for data exports from projects with a large number of variables
- Case insensitive

In [ ]:
def find_col(df, subs):
    return list(filter(lambda x: subs.lower() in x.lower(), df.columns))

find_col(post_rand_df, 'ex')

## Armed with the Data Dictionary, let's mark forms as complete in Redcap via the API

### Review the latest export

In [ ]:
new_df

### First, reset the max number of DataFrame columns and rows displayed in notebook

In [ ]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

### Every Redcap form has a hidden completion variable 
- Form name, separated by underscore, and appended with '_complete'
- It is possible that a variable was named with the same suffix, so there is no guarantee that a field with this suffix is in fact the form completion variable

In [ ]:
complete_fields = find_col(new_df, '_complete')  # in this example, these are all form completion fields
complete_fields

### Working assumptions 
- All forms we have worked with should have been marked complete
- Except for randomization, where anyone not randomized is left as is

In [ ]:
list(map(lambda x: x.split('_complete')[0], dd_df.form_name.unique()))

In [ ]:
new_df

In [ ]:
# Define a helper function to use for each row of the DataFrame
def check_completion(roe, f):
#     set_trace()
    # First, validate the field name against the dictionary by seeing if the prefix is a form
    if f.split('_complete')[0] in list(map(lambda x: x, dd_df.form_name.unique())):
        if roe[f] == '0' and f != 'randomization_complete':
            return '2' 
        elif roe[f] == '0' and f == 'randomization_complete' and roe['rand_date']:
            return '2'
        else:
            return roe[f]
    else:
        roe[f]
        
# test_df = new_df.copy(deep=True)
for field in complete_fields:
    new_df[field] = new_df.apply(lambda row: check_completion(row, field), axis=1)

new_df

In [ ]:
core_field_list = ['redcap_event_name', 'redcap_repeat_instrument', 'redcap_repeat_instance', 'record_id']
complete_df = new_df[core_field_list + complete_fields].copy(deep=True)
complete_df


### Import the completion fields and review Dashboard in Redcap

In [ ]:
success_count, total_count = import_to_redcap(rci, complete_df)
success_count, total_count, success_count == total_count

## Import a file
- Display in notebook
- Ref: https://stackoverflow.com/questions/32370281/how-to-include-image-or-picture-in-jupyter-notebook

In [ ]:
from IPython import display
from IPython.core.display import HTML 

img_url = 'https://image.shutterstock.com/z/stock-vector-cartoon-tooth-giving-thumb-up-602566124.jpg'
display.Image(url=img_url, width=200, height=200)

### Store image to file
- Ref: http://www.effbot.org/imagingbook/introduction.htm

In [ ]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get(img_url)
img = Image.open(BytesIO(response.content))

img_filename = 'happy_tooth.jpg'
img.save(img_filename, 'JPEG')
img.show()  # Opens an external application

### Create a column containing the filename
- Depending on your environment, you may want to use a full path

In [ ]:
new_df['exam_photo'] = new_df.exam_complete.apply(lambda x: img_filename if x == '2' else '')
new_df

### Import the files
- Redcap has a delete file API endpoint, but it is not yet implemented in Redcapy
- A Redcap variable is associated with a single file, so a different file will overwrite an existing file
- Redcap should be able to accept any type of file, but only html and image files have been tested

In [ ]:
import os
import redcap.redcapy

def do_file(action):
    """
        :param action: str, Values are import/delete
    """
    valid_actions = ['import', 'delete']
    
    if action not in valid_actions:
        raise ValueError('Invalid action argument passed.  Must be one of {}'.format(', '.join(valid_actions)))
    
    returned_list = []
    
    for i, row in new_df.iterrows():
        if os.path.exists(row.exam_photo):
            filename = row.exam_photo if action == 'import' else ''
            returned = rci.import_file(event = row.redcap_event_name,
                                       field = 'exam_photo', 
                                       filename = filename, 
                                       record_id = row.record_id,
                                       repeat_instance = '',
                                       action = action,
                                      )
            returned_list.append(returned)
            
    return returned_list
    
result = do_file('import')
result, sum(result)

### Delete the files from Redcap that were just imported 

In [ ]:
result = do_file('delete')
result, sum(result)

## Exporting Data Efficiently
- For projects with large amounts of data (fields and/or participants), economize use of the export API
 - Implement local caching of data
 - Limit export to select fields or events
  - Redcapy will have a record id option in a future update
   - This will allow you to cache an object containing record_ids to exclude from data exports, or write to a flat file
    - One implementation idea: 
     - As IDs meet a permanent condition, write record_id to exclusion object
     - When launching code, first download a full list of record_ids.
     - Open and check exclusion list against the full list of ids.
     - Export a difference of all IDs and exlcusions.
    - The export efficiency may be nominal at first, but can become substantial as the project accrues more participants.
- Start a notebook cell with %%time command to evaluate how long it takes to execute the cell
 - A full list of available magic commands can be viewed by executing %magic in a cell

In [ ]:
?%time

### Caching
- Jupyter notebooks can use the %store magic command
 - Ref: https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/
 - %store also allows you to access stored objects from any notebook
 - Storage is persistent on disk
 - Stored objects are located (your results may vary) in:
   - ~/.ipython/profile_default/db/autorestore
 - Object names should be unique, as an identically named object from one notebook can overwrite one from another
- Python programs can use pickle
 -  Ref: https://www.thoughtco.com/using-pickle-to-save-objects-2813661


#### %store Magic command

In [ ]:
use_cached_data = False

In [ ]:
if not use_cached_data:
    caps_data_export = rce.export_records(rawOrLabel='raw')
    caps_dd_export = rce.export_data_dictionary()

    # Store data for future use to avoid API calls
    %store caps_data_export
    %store caps_dd_export
else:  # restore prior data
    %store -r caps_data_export 
    %store -r caps_dd_export 

#### Limiting export by field or event
- Specify events by their raw version 
 - e.g., 'baseline_arm_1', not 'Baseline (Arm 1: Baseline)'

In [ ]:
baseline_data = rce.export_records(events='baseline_arm_1')
baseline_data

In [ ]:
%time
baseline_data = rce.export_records(events='baseline_arm_1')
# baseline_data

#### Filter further by field name
- Use a comma separated string to extract multiple events or fields

In [ ]:
baseline_names = rce.export_records(events='baseline_arm_1', fields='part_first_name, part_last_name')
baseline_names

### Export the number of teeth from exams

In [ ]:
post_baseline_teeth = rce.export_records(events='6_month_arm_2, 6_month_arm_3, 12_month_arm_2, 12_month_arm_3', 
                                         fields='exam_num_teeth')
post_baseline_teeth

### Now add record_id to the list of fields
- Note the change in data returned

In [ ]:
post_baseline_teeth = rce.export_records(events='6_month_arm_2, 6_month_arm_3, 12_month_arm_2, 12_month_arm_3', 
                                         fields='record_id, exam_num_teeth')
post_baseline_teeth

## Using labels and raw data
- Python does not natively offer the same convenience as SAS for handling labeled data, as with SAS formats
- However we can improvise by using a mix of raw and labeled data exports
- During a project in production, consider the likelihood of whether your raw or labeled values may change

In [ ]:
raw_export = rce.export_records(rawOrLabel='raw')
label_export = rce.export_records(rawOrLabel='label')

raw_df = pd.DataFrame.from_records(raw_export)
label_df = pd.DataFrame.from_records(label_export)

raw_df.shape, label_df.shape, 

### View labeled data

In [ ]:
label_df.head()

### View raw equivalent

In [ ]:
raw_df.head()

### A trick is to alter the variable names of the labeled variable names
- For example, you could perform calculations using the raw values and display the labeled values of an ordinal variable
- Combine both raw and labeled DataFrames into a single DataFrame, appending a '_label' suffix to the column name

In [ ]:
def join_dfs(dfraw, dflabel):
    rc_keys = ['record_id', 'redcap_repeat_instance']  # These are identical in both DataFrames
    dflabel2 = dflabel.copy(deep=True).rename(columns={col: '{}_label'.format(col) 
                                                       for col in dflabel.columns 
                                                       if col not in rc_keys})
    df = pd.concat([dfraw, dflabel2], levels=rc_keys, axis=1)
    
    # Remove columns with dup col names, Ref: https://stackoverflow.com/a/16939512
    df = df.T.groupby(level=0).first().T  
    
    return df
   
combined_df = join_dfs(raw_df, label_df)
combined_df.columns

In [ ]:
combined_df

In [ ]:
grades = combined_df.pfu_ae_grade.apply(lambda x: int(x) if x.isdigit() else np.NaN)
grades

### Table of IDs and Grades where the Grade > Mean Grade
#### First create a filter for the DataFrame to extract rows corresponding to mask rows == True

In [ ]:
mask = grades > grades.mean()
mask

#### Next, apply the mask to the DataFrame, and specify a list of columns of interest

In [ ]:
combined_df[mask][['record_id', 'pfu_ae_grade_label']]

### Simple filters can be more easily written as a one-liner, but readability improves as a separate line for more complex filters

In [ ]:
combined_df[grades > grades.mean()][['record_id', 'pfu_ae_grade_label']]

### Typically, you may work directly with DataFrame columns to use as a filter
- For a series that requires a transformation like pfu_ae_grade, you might assign the transformed series to a DataFrame as a column

In [ ]:
combined_df['pfu_ae_grade_num'] = combined_df.pfu_ae_grade.apply(lambda x: int(x) if x.isdigit() 
                                                                else np.NaN)
combined_df[combined_df.pfu_ae_grade_num > combined_df.pfu_ae_grade_num.mean()][['record_id', 'pfu_ae_grade_label']]

### If you have a series of variables requiring a similar computation

In [ ]:
def compute_above_mean(df, field):
    num_name = field + '_num'
    df[num_name] = df[field].apply(lambda x: int(x) if x.isdigit() else np.NaN)

    return df[df[num_name] > df[num_name].mean()][['record_id', field + '_label']]
    
compute_above_mean(combined_df, 'pfu_ae_grade')

In [ ]:
compute_above_mean(combined_df, 'pfu_ae_serious')

## Working with import overwrite behavior
- Null data elements should not overwrite extant data in Redcap by default
- In the Redcapy import_data method, the parameter overwriteBehavior == 'normal' by default
 - Setting overwriteBehavior='overwrite' will overwrite Redcap data with blanks
 - In a worst case scenario, if you begin with a DataFrame scaffold of all fields, populate some fields of interest, and import with overwrite set to 'overwrite' it will erase all other fields corresponding to those IDs and events.
- Normally, you will want to use the default

In [ ]:
# rci.import_records(data_to_upload=data_to_import, overwriteBehavior='normal')  # default
# rci.import_records(data_to_upload=data_to_import, overwriteBehavior='overwrite') 

### Store this DataFrame for use in the next notebook
- Creata a name that is unique across all notebooks

In [ ]:
caps_combined_df = combined_df
%store caps_combined_df